In [1]:
import pandas as pd
import pysam
import numpy as np
import akita_utils
import h5py

import bioframe as bf

In [2]:
from akita_utils.dna_utils import scan_motif, dna_1hot
from akita_utils.format_io import read_jaspar_to_numpy

2023-06-05 14:22:38.895124: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home1/smaruj/software/GSL/lib:/home1/smaruj/software/HTSLIB/lib
2023-06-05 14:22:38.895167: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
genome_file = "/project/fudenber_735/genomes/mm10/mm10.fa"

# reading tsv with all 1000 motifs

In [4]:
tsv_filepath = "/home1/smaruj/akita_utils/bin/insert_virtual_dots_vs_boundaries/boundaries_10k.tsv"
df = pd.read_table(tsv_filepath, sep="\t")

In [5]:
# adding seq_id

seq_id = []

nr_targets = 6
nr_sites = 1000
nr_backgrounds = 10
# since there are 1000 motifs and 10 backgrounds

for background_index in range(nr_backgrounds):
    for seq_index in range(nr_sites):
        seq_id.append(seq_index)

In [6]:
df["seq_id"] = seq_id

In [7]:
df_bg0 = df[df["background_index"] == 0]

In [8]:
# since the set of motifs is the same for all backgrounds, I'd focus on a subset of the table corresponding to the 0th background
df_bg0

,chrom,start,end,strand,genomic_SCD,orientation,background_index,flank_bp,spacer_bp,seq_id
0,chr12,35192361,35192380,+,53.780,<>,0,20,99980,0
1,chr2,48886252,48886271,-,53.700,<>,0,20,99980,1
2,chr12,35192359,35192378,-,53.660,<>,0,20,99980,2
3,chr8,102781112,102781131,-,53.500,<>,0,20,99980,3
4,chr5,49961991,49962010,-,53.300,<>,0,20,99980,4
...,...,...,...,...,...,...,...,...,...,...
995,chr2,118246115,118246134,-,4.390,<>,0,20,99980,995
996,chr5,117502801,117502820,+,4.367,<>,0,20,99980,996
997,chr15,76198798,76198817,+,4.363,<>,0,20,99980,997
998,chr5,143315378,143315397,-,4.360,<>,0,20,99980,998


In [ ]:
# 1000 strongest CTCF-binding sites based on:
# /project/fudenber_735/tensorflow_models/akita/v2/analysis/permute_boundaries_motifs_ctcf_mm10_model*/scd.h5

In [9]:
# reading CTCF motif from the jaspar file
motif_matrix = read_jaspar_to_numpy()

In [10]:
def get_forward_score(s):
    
    motif_matrix = read_jaspar_to_numpy()
    dna = dna_1hot(genome_open.fetch(s.chrom, s.start - s.flank_bp, s.end + s.flank_bp).upper())
    
    max_forward_score = scan_motif(dna, motif_matrix, strand="forward").max()
    
    return max_forward_score
    

In [11]:
def get_reverse_score(s):
    
    motif_matrix = read_jaspar_to_numpy()
    dna = dna_1hot(genome_open.fetch(s.chrom, s.start - s.flank_bp, s.end + s.flank_bp).upper())
    
    max_reverse_score = scan_motif(dna, motif_matrix, strand="reverse").max()
    
    return max_reverse_score
    

In [12]:
def get_scan_strand(s):
    
    if s.forward_scan >= s.reverse_scan:
        return "+"
    else:
        return "-"

In [13]:
def check_mismatch(s):
    if s.strand != s.scan_strand:
        return 1
    else:
        return 0


In [14]:
def get_rf_diff(s):
    diff = s.forward_scan - s.reverse_scan
    if diff >= 0:
        return diff
    else:
        return -1 * diff

In [16]:
genome_open = pysam.Fastafile(genome_file)

In [17]:
df_bg0.loc[:, ("forward_scan")] = df_bg0.apply(get_forward_score, axis=1)
df_bg0.loc[:, ("reverse_scan")] = df_bg0.apply(get_reverse_score, axis=1)

2023-06-05 14:23:07.807229: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home1/smaruj/software/GSL/lib:/home1/smaruj/software/HTSLIB/lib
2023-06-05 14:23:07.807301: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-05 14:23:07.807366: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (d11-06.hpc.usc.edu): /proc/driver/nvidia/version does not exist
2023-06-05 14:23:07.808181: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/tmp/SLURM_1

In [18]:
df_bg0.loc[:, ("scan_strand")] = df_bg0.apply(get_scan_strand, axis=1)
df_bg0.loc[:, ("mismatch")] = df_bg0.apply(check_mismatch, axis=1)

/tmp/SLURM_13544149/ipykernel_86069/2158644260.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bg0.loc[:, ("scan_strand")] = df_bg0.apply(get_scan_strand, axis=1)
/tmp/SLURM_13544149/ipykernel_86069/2158644260.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bg0.loc[:, ("mismatch")] = df_bg0.apply(check_mismatch, axis=1)


In [19]:
df_bg0.loc[:, ("rf_diff")] = df_bg0.apply(get_rf_diff, axis=1)

/tmp/SLURM_13544149/ipykernel_86069/189930804.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bg0.loc[:, ("rf_diff")] = df_bg0.apply(get_rf_diff, axis=1)


In [20]:
df_bg0

,chrom,start,end,strand,genomic_SCD,orientation,background_index,flank_bp,spacer_bp,seq_id,forward_scan,reverse_scan,scan_strand,mismatch,rf_diff
0,chr12,35192361,35192380,+,53.780,<>,0,20,99980,0,9.791526,11.551709,-,1,1.760183
1,chr2,48886252,48886271,-,53.700,<>,0,20,99980,1,6.718716,11.075207,-,0,4.356491
2,chr12,35192359,35192378,-,53.660,<>,0,20,99980,2,9.791526,11.551709,-,0,1.760183
3,chr8,102781112,102781131,-,53.500,<>,0,20,99980,3,9.260164,11.769504,-,0,2.509340
4,chr5,49961991,49962010,-,53.300,<>,0,20,99980,4,8.901366,11.003430,-,0,2.102064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,chr2,118246115,118246134,-,4.390,<>,0,20,99980,995,8.800995,11.633396,-,0,2.832402
996,chr5,117502801,117502820,+,4.367,<>,0,20,99980,996,11.306763,8.516800,+,0,2.789963
997,chr15,76198798,76198817,+,4.363,<>,0,20,99980,997,11.014260,9.274653,+,0,1.739608
998,chr5,143315378,143315397,-,4.360,<>,0,20,99980,998,8.716151,11.336373,-,0,2.620223


# Task 1    
### bioframe.overlap(42 sites, jaspar bedfile for positions): do they have the same strand or did it get mixed up?

In [21]:
mismatch_df = df_bg0[df_bg0["mismatch"] == 1]

In [22]:
mismatch_df

,chrom,start,end,strand,genomic_SCD,orientation,background_index,flank_bp,spacer_bp,seq_id,forward_scan,reverse_scan,scan_strand,mismatch,rf_diff
0,chr12,35192361,35192380,+,53.780,<>,0,20,99980,0,9.791526,11.551709,-,1,1.760183
54,chr4,74231658,74231677,+,45.220,<>,0,20,99980,54,10.327698,11.429803,-,1,1.102105
66,chr5,16309780,16309799,-,43.750,<>,0,20,99980,66,10.550266,10.283763,+,1,0.266503
97,chr17,62884328,62884347,+,41.220,<>,0,20,99980,97,9.643594,10.292944,-,1,0.649349
128,chr13,113639041,113639060,-,38.000,<>,0,20,99980,128,11.921696,10.328792,+,1,1.592904
144,chr8,90793952,90793971,+,36.250,<>,0,20,99980,144,9.833800,11.627345,-,1,1.793545
152,chr3,79202323,79202342,-,35.500,<>,0,20,99980,152,10.178311,9.898915,+,1,0.279396
191,chr9,74871152,74871171,+,31.670,<>,0,20,99980,191,9.612469,11.063319,-,1,1.450850
195,chr3,67352554,67352573,-,31.360,<>,0,20,99980,195,11.759932,9.775270,+,1,1.984662
256,chr1,182991110,182991129,-,27.670,<>,0,20,99980,256,11.459613,10.665792,+,1,0.793821


In [23]:
mismatch_df["rf_diff"].mean()

1.1120956329421696

In [24]:
samples = 10
average = 0.0

counter = 0
while counter <= 10:
    match_df = df_bg0[df_bg0["mismatch"] == 0]
    sample_match_df = match_df.sample(n=42, replace=False)
    average += sample_match_df["rf_diff"].mean()
    counter += 1
    
average = average / samples
average

2.6710498750351723

**Observation**: the differences between forward_scan and reverse_scan are relatively smaller for the sequences with mismatched strands

**Question**: The "strand" column (coordinates of all CTCF-binding sites actually) is taken from the jasper file (downloaded from the jaspar database)    
Let's check which strand column in the jaspar tsv

In [25]:
jaspar_file = "/project/fudenber_735/motifs/mm10/jaspar/MA0139.1.tsv.gz"

In [29]:
jaspar_df = bf.read_table(jaspar_file, schema="jaspar")

In [30]:
jaspar_df

,chrom,start,end,name,score,pval,strand
0,chr10,3103662,3103681,CTCF,814,410,-
1,chr10,3108121,3108140,CTCF,859,504,-
2,chr10,3119020,3119039,CTCF,801,386,-
3,chr10,3124081,3124100,CTCF,807,397,-
4,chr10,3127081,3127100,CTCF,810,403,+
...,...,...,...,...,...,...,...
825505,chrY,90829177,90829196,CTCF,842,465,+
825506,chrY,90839573,90839592,CTCF,804,392,+
825507,chrY,90839785,90839804,CTCF,885,565,-
825508,chrY,90839787,90839806,CTCF,842,465,+


In [31]:
jaspar_overlap = bf.overlap(mismatch_df, jaspar_df, how="left", suffixes=("_mis","_jaspar"))

In [33]:
jaspar_overlap.head(20)

,chrom_mis,start_mis,end_mis,strand_mis,genomic_SCD_mis,orientation_mis,background_index_mis,flank_bp_mis,spacer_bp_mis,seq_id_mis,...,scan_strand_mis,mismatch_mis,rf_diff_mis,chrom_jaspar,start_jaspar,end_jaspar,name_jaspar,score_jaspar,pval_jaspar,strand_jaspar
0,chr12,35192361,35192380,+,53.78,<>,0.0,20.0,99980.0,0.0,...,-,1.0,1.760183,chr12,35192359,35192378,CTCF,931.0,699.0,-
1,chr12,35192361,35192380,+,53.78,<>,0.0,20.0,99980.0,0.0,...,-,1.0,1.760183,chr12,35192361,35192380,CTCF,805.0,393.0,+
2,chr4,74231658,74231677,+,45.22,<>,0.0,20.0,99980.0,54.0,...,-,1.0,1.102105,chr4,74231656,74231675,CTCF,928.0,688.0,-
3,chr4,74231658,74231677,+,45.22,<>,0.0,20.0,99980.0,54.0,...,-,1.0,1.102105,chr4,74231658,74231677,CTCF,864.0,513.0,+
4,chr5,16309780,16309799,-,43.75,<>,0.0,20.0,99980.0,66.0,...,+,1.0,0.266503,chr5,16309780,16309799,CTCF,833.0,448.0,-
5,chr5,16309780,16309799,-,43.75,<>,0.0,20.0,99980.0,66.0,...,+,1.0,0.266503,chr5,16309782,16309801,CTCF,875.0,540.0,+
6,chr17,62884328,62884347,+,41.22,<>,0.0,20.0,99980.0,97.0,...,-,1.0,0.649349,chr17,62884325,62884344,CTCF,846.0,473.0,-
7,chr17,62884328,62884347,+,41.22,<>,0.0,20.0,99980.0,97.0,...,-,1.0,0.649349,chr17,62884328,62884347,CTCF,802.0,387.0,+
8,chr13,113639041,113639060,-,38.00,<>,0.0,20.0,99980.0,128.0,...,+,1.0,1.592904,chr13,113639041,113639060,CTCF,830.0,440.0,-
9,chr13,113639041,113639060,-,38.00,<>,0.0,20.0,99980.0,128.0,...,+,1.0,1.592904,chr13,113639044,113639063,CTCF,946.0,752.0,+


In [34]:
jaspar_overlap.tail(10)

,chrom_mis,start_mis,end_mis,strand_mis,genomic_SCD_mis,orientation_mis,background_index_mis,flank_bp_mis,spacer_bp_mis,seq_id_mis,...,scan_strand_mis,mismatch_mis,rf_diff_mis,chrom_jaspar,start_jaspar,end_jaspar,name_jaspar,score_jaspar,pval_jaspar,strand_jaspar
78,chr18,37727961,37727980,+,5.230,<>,0.0,20.0,99980.0,955.0,...,-,1.0,0.891325,chr18,37727959,37727978,CTCF,872.0,533.0,-
79,chr18,37727961,37727980,+,5.230,<>,0.0,20.0,99980.0,955.0,...,-,1.0,0.891325,chr18,37727961,37727980,CTCF,820.0,421.0,+
80,chr18,37727961,37727980,+,5.230,<>,0.0,20.0,99980.0,955.0,...,-,1.0,0.891325,chr18,37727964,37727983,CTCF,807.0,397.0,+
81,chr1,9541481,9541500,+,4.680,<>,0.0,20.0,99980.0,983.0,...,-,1.0,0.160944,chr1,9541468,9541487,CTCF,862.0,510.0,-
82,chr1,9541481,9541500,+,4.680,<>,0.0,20.0,99980.0,983.0,...,-,1.0,0.160944,chr1,9541481,9541500,CTCF,819.0,420.0,+
83,chr2,9889394,9889413,+,4.547,<>,0.0,20.0,99980.0,989.0,...,-,1.0,0.147352,chr2,9889392,9889411,CTCF,875.0,541.0,-
84,chr2,9889394,9889413,+,4.547,<>,0.0,20.0,99980.0,989.0,...,-,1.0,0.147352,chr2,9889394,9889413,CTCF,858.0,500.0,+
85,chr4,125990358,125990377,+,4.484,<>,0.0,20.0,99980.0,992.0,...,-,1.0,2.083960,chr4,125990342,125990361,CTCF,979.0,921.0,-
86,chr4,125990358,125990377,+,4.484,<>,0.0,20.0,99980.0,992.0,...,-,1.0,2.083960,chr4,125990355,125990374,CTCF,814.0,410.0,+
87,chr4,125990358,125990377,+,4.484,<>,0.0,20.0,99980.0,992.0,...,-,1.0,2.083960,chr4,125990358,125990377,CTCF,822.0,426.0,+


Now, the "strand_mis" column is the strand we had initially in the tsv file, while the "strand_jaspar" column is the strand in the jaspar tsv.   

**Note**, in the jaspar file there are two rows with different strands corresponding to one row in our tsv. Coordinates differ by a few bps. This means that the scanning from both directions give scores above threshold and both motifs are reported, although our tsv has information only about one of them (likely, the weaker one).   

**Question**, however, there is additional step of filtering out overlapping CTCF-binding sites. Why those CTCFs are not filtered out if they obviously overlap and the strand column does not matter in the filtering out procedure.


# What is wrong with the filtering out step?

## 1. Checking filter_by_ctcf()

In [45]:
from io import StringIO

In [46]:
# from akita_utils.tsv_gen_utils import filter_by_ctcf

In [50]:
# filter_by_ctcf

sites = df_bg0.rename(columns={"start": "start_2", "end": "end_2"})
ctcf_file="/project/fudenber_735/motifs/mm10/jaspar/MA0139.1.tsv.gz"
exclude_window=60
site_cols=["chrom", "start", "end"]
verbose=True

# if verbose:
#     print("filtering sites by overlap with ctcfs")

ctcf_cols = list(
    pd.read_csv(
        StringIO("""chrom start end name score pval strand"""),
        sep=" ",
    )
)

ctcf_motifs = pd.read_table(
    ctcf_file,
    names=ctcf_cols,
)

ctcf_motifs = bf.expand(ctcf_motifs, pad=exclude_window)

# ctcf_motifs

sites = bf.count_overlaps(
    sites, ctcf_motifs[site_cols], cols1=["chrom", "start_2", "end_2"]
)

sites = sites.iloc[sites["count"].values == 1]
sites.reset_index(inplace=True, drop=True)

sites

,chrom,start_2,end_2,strand,genomic_SCD,orientation,background_index,flank_bp,spacer_bp,seq_id,forward_scan,reverse_scan,scan_strand,mismatch,rf_diff,count
0,chr2,48886252,48886271,-,53.700,<>,0,20,99980,1,6.718716,11.075207,-,0,4.356491,1
1,chr8,102781112,102781131,-,53.500,<>,0,20,99980,3,9.260164,11.769504,-,0,2.509340,1
2,chr5,49961991,49962010,-,53.300,<>,0,20,99980,4,8.901366,11.003430,-,0,2.102064,1
3,chr9,26776356,26776375,-,53.120,<>,0,20,99980,5,9.089728,11.181056,-,0,2.091328,1
4,chr9,51152589,51152608,-,53.060,<>,0,20,99980,6,8.523872,10.950586,-,0,2.426715,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,chr19,8718712,8718731,-,4.555,<>,0,20,99980,988,7.587506,9.341849,-,0,1.754344,1
821,chr19,34989434,34989453,+,4.438,<>,0,20,99980,993,10.438280,8.464608,+,0,1.973672,1
822,chr2,118246115,118246134,-,4.390,<>,0,20,99980,995,8.800995,11.633396,-,0,2.832402,1
823,chr15,76198798,76198817,+,4.363,<>,0,20,99980,997,11.014260,9.274653,+,0,1.739608,1


## 2. Checking filter_by_overlap_num

In [58]:
def filter_by_overlap_num(
    working_df,
    filter_df,
    expand_window=60,
    working_df_cols=["chrom","start","end"],
    filter_df_cols=["chrom","start","end"],
    max_overlap_num=0):
    
    """
    Filter out rows from working_df that overlap entries in filter_df above given threshold.

    Parameters
    -----------
    working_df : dataFrame
        First set of genomic intervals.
    filter_df : dataFrame
        Second set of genomic intervals.
    expand_window : int
        Indicates how big window around the given genomic intervals should be taken into account.
    working_df_cols : list
        Columns specifying genomic intervals in the working_df.
    filter_df_cols : list
        Columns specifying genomic intervals in the filter_df.
    max_overlap_num : int
        All the rows with number of overlaps above this threshold will be filtered out.
        
    Returns
    --------
    working_df : dataFrame
        Subset of working_df that do not have overlaps with filter_df above given threshold.

    """
    
    filter_df = bf.expand(filter_df, pad=expand_window)
    
    working_df = bf.count_overlaps(working_df, filter_df[filter_df_cols], cols1=working_df_cols)
    
    working_df = working_df.iloc[working_df["count"].values <= max_overlap_num]
    working_df.reset_index(inplace=True, drop=True)

    return working_df

In [59]:
filter_by_overlap_num(df_bg0,
                    filter_df=jaspar_df,
                     max_overlap_num=1)

,chrom,start,end,strand,genomic_SCD,orientation,background_index,flank_bp,spacer_bp,seq_id,forward_scan,reverse_scan,scan_strand,mismatch,rf_diff,count
0,chr2,48886252,48886271,-,53.700,<>,0,20,99980,1,6.718716,11.075207,-,0,4.356491,1
1,chr8,102781112,102781131,-,53.500,<>,0,20,99980,3,9.260164,11.769504,-,0,2.509340,1
2,chr5,49961991,49962010,-,53.300,<>,0,20,99980,4,8.901366,11.003430,-,0,2.102064,1
3,chr9,26776356,26776375,-,53.120,<>,0,20,99980,5,9.089728,11.181056,-,0,2.091328,1
4,chr9,51152589,51152608,-,53.060,<>,0,20,99980,6,8.523872,10.950586,-,0,2.426715,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,chr19,8718712,8718731,-,4.555,<>,0,20,99980,988,7.587506,9.341849,-,0,1.754344,1
821,chr19,34989434,34989453,+,4.438,<>,0,20,99980,993,10.438280,8.464608,+,0,1.973672,1
822,chr2,118246115,118246134,-,4.390,<>,0,20,99980,995,8.800995,11.633396,-,0,2.832402,1
823,chr15,76198798,76198817,+,4.363,<>,0,20,99980,997,11.014260,9.274653,+,0,1.739608,1


It looks that both functions work as expected!

# Task 2    
### look at profiles for (some of) the 42 sites: do they have reasonable maximas on both strands?

In [ ]:
mismatch_df

In [ ]:
# test
chrom = "chr12"
start = 35192361
end = 35192380

In [ ]:
def get_forward_score(s):
    
    motif_matrix = read_jaspar_to_numpy()
    dna = dna_1hot(genome_open.fetch(s.chrom, s.start - s.flank_bp, s.end + s.flank_bp).upper())
    
    max_forward_score = scan_motif(dna, motif_matrix, strand="forward")
    
    
    return max_forward_score

In [ ]:
def get_reverse_score(s):
    
    motif_matrix = read_jaspar_to_numpy()
    dna = dna_1hot(genome_open.fetch(s.chrom, s.start - s.flank_bp, s.end + s.flank_bp).upper())
    
    max_reverse_score = scan_motif(dna, motif_matrix, strand="reverse").max()
    
    return max_reverse_score

In [ ]:
scan_strand_list_for_df = []

for i in range(nr_backgrounds):
    scan_strand_list_for_df = scan_strand_list_for_df + list(df_bg0["scan_strand"])

In [ ]:
mismatch_list_for_df = []

for i in range(nr_backgrounds):
    mismatch_list_for_df = mismatch_list_for_df + list(df_bg0["mismatch"])

In [ ]:
df["mismatch"] = mismatch_list_for_df

In [ ]:
df["scan_strand"] = scan_strand_list_for_df

In [ ]:
df

In [ ]:
df.to_csv("scanned_boundaries_10k.tsv", sep="\t")